In [1]:
import sys
import os
import glob

# Append the DaVinci Resolve scripting module path
sys.path.append('/Library/Application Support/Blackmagic Design/DaVinci Resolve/Developer/Scripting/Modules')
import DaVinciResolveScript as dvr_script

# Initialize DaVinci Resolve
resolve = dvr_script.scriptapp("Resolve")

# Get the project manager and current project
projectManager = resolve.GetProjectManager()
currentProject = projectManager.GetCurrentProject()
media_pool = currentProject.GetMediaPool()

currentTimeline = currentProject.GetCurrentTimeline()
if currentTimeline is None:
    currentTimeline = media_pool.CreateEmptyTimeline("MyTimeLine")

print(f"Current project: {currentProject.GetName()}")
print(f"Current timeline: {currentTimeline.GetName()}")

Current project: Untitled Project 4
Current timeline: MyTimeLine


In [2]:
# Path to the Taps sound folder in Downloads
taps_folder = os.path.expanduser("~/Downloads/Taps")

# Find all .wav files in the folder, sorted by name
sound_files = sorted(glob.glob(os.path.join(taps_folder, "tap_*.wav")))

print(f"Found {len(sound_files)} sound files:")
for f in sound_files:
    print(f"  {os.path.basename(f)}")

Found 9 sound files:
  tap_001.wav
  tap_002.wav
  tap_003.wav
  tap_004.wav
  tap_005.wav
  tap_006.wav
  tap_007.wav
  tap_008.wav
  tap_009.wav


In [3]:
# Import all sound files into the media pool
imported = media_pool.ImportMedia(sound_files)

if imported:
    print(f"Successfully imported {len(imported)} sound files into the media pool.")
else:
    print("Failed to import sound files.")

Successfully imported 9 sound files into the media pool.


In [10]:
# Find the imported sound clips in the media pool
root_folder = media_pool.GetRootFolder()
all_clips = root_folder.GetClipList()

# Match imported sound clips by filename
sound_basenames = {os.path.basename(f) for f in sound_files}
sound_clips = []

for clip in all_clips:
    clip_name = clip.GetName()
    if clip_name in sound_basenames:
        sound_clips.append(clip)

# Sort clips by name to keep tap_001 -> tap_009 order
sound_clips.sort(key=lambda c: c.GetName())

# Get timeline frame rate and start frame
fps = int(currentTimeline.GetSetting("timelineFrameRate"))
start_frame = currentTimeline.GetStartFrame()
print(f"Timeline FPS: {fps}, Start frame: {start_frame}")

# Place each clip at sec 1, 3, 5, 7, 9, 11, 13, 15, 17
for i, clip in enumerate(sound_clips):
    target_second = 1 + (i * 4)  # sec 1, 5, 9, 13, ...
    record_frame = start_frame + (target_second * fps)
    
    # Alternate between audio track 1 and 2
    track_index = 1 if i % 2 == 0 else 2

    clip_info = {
        "mediaPoolItem": clip,
        "recordFrame": record_frame,
        "trackIndex": track_index,
    }
    
    result = media_pool.AppendToTimeline([clip_info])
    status = "OK" if result else "FAILED"
    print(f"  {clip.GetName()} -> sec {target_second}, track {track_index} (frame {record_frame}) [{status}]")

print(f"\nDone! Placed {len(sound_clips)} sound clips at 2-second intervals.")

Timeline FPS: 24, Start frame: 86400
  tap_001.wav -> sec 1, track 1 (frame 86424) [OK]
  tap_002.wav -> sec 5, track 2 (frame 86520) [OK]
  tap_003.wav -> sec 9, track 1 (frame 86616) [OK]
  tap_004.wav -> sec 13, track 2 (frame 86712) [OK]
  tap_005.wav -> sec 17, track 1 (frame 86808) [OK]
  tap_006.wav -> sec 21, track 2 (frame 86904) [OK]
  tap_007.wav -> sec 25, track 1 (frame 87000) [OK]
  tap_008.wav -> sec 29, track 2 (frame 87096) [OK]
  tap_009.wav -> sec 33, track 1 (frame 87192) [OK]

Done! Placed 9 sound clips at 2-second intervals.
